In [1]:
"""
File:          Feature_Extraction.py
Author:        Alex Cui
Last Modified:
"""

import numpy as np
import cv2
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib import cm
from feature_extraction import Feature_Extraction

class Feature_Extraction:

    def __init__(self, img):
        """Initializes feature extraction elements by 
        taking in the path of the image to be processed
        """
        self.image = cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2RGB)
        self.scale_percent = 20
        self.detector = cv2.ORB_create(nfeatures=100000, scoreType=cv2.ORB_FAST_SCORE)
        self.yellow_lower = np.array([22,60,200],np.uint8)
        self.yellow_upper = np.array([60,255,255],np.uint8)

    def denoise_real_image(self):
        """Denoise the image before running feature extraction
        """

        #resize image to the value of scale_percent
        width = int(self.image.shape[1] * self.scale_percent / 100)
        height = int(self.image.shape[0] * self.scale_percent / 100)
        dim = (width, height)
        self.image = cv2.resize(self.image, dim, interpolation = cv2.INTER_AREA)

        #Denoise the image
        self.image = cv2.fastNlMeansDenoisingColored(self.image, None, 7, 21, 50, 10)

    def ignore_above(self):
        """Ignore noises outside the field
        """ 

        mask = self.__compute_mask()
        x = self.__get_highest(mask)
        mask[x:, :] = 255
        mask[:x, :] = 0
        self.image = cv2.bitwise_and(self.image, self.image, mask = mask)

    def __compute_mask(self):
        """Private method that computes mask
        """

        #Create yellow_mask
        hsv = cv2.cvtColor(self.image, cv2.COLOR_RGB2HSV)
        yellow_mask = cv2.inRange(hsv, self.yellow_lower, self.yellow_upper)
        return yellow_mask

    def __get_highest(self, mask):
        """Private method that computes the height above which 
        everything is ignored
        """
        for i in range (mask.shape[0]):
            s = np.sum(mask[i])
            if (s >= 255 * mask.shape[1]):
                return i

    def detect_features(self):
        """Detect features
        """       
        # find keypoints
        kp = self.detector.detect(self.image, None)
        kp, des = self.detector.compute(self.image, kp)
        k = cv2.drawKeypoints(self.image, kp, None, color=(0,0,255))
        self.image = k

ModuleNotFoundError: No module named 'feature_extraction'

In [ ]:
r_path = '../PF_Images/real_images_input/r_3.png'
s_path = '../PF_Images/simulated_images_input/s_3.png'

In [ ]:
im1 = Feature_Extraction(path1)
im1.image = im1.image[60: 410, 150: 510]
im2 = Feature_Extraction(path2)
im2.image = im2.image[60: 410, 150: 510]
im3 = Feature_Extraction(path3)
im3.image = im3.image[60: 410, 150: 510]

In [ ]:
orb = cv2.ORB_create()

kp1 = orb.detect(im2.image, None)
kp1, des1 = orb.compute(im2.image, kp1)

kp2 = orb.detect(im3.image, None)
kp2, des2 = orb.compute(im3.image, kp2)

if type(des1) != np.float32:
    des1 = np.float32(des1)
    
if type(des2) != np.float32:
    des2 = np.float32(des2)

FLANN_INDEX_KDTREE = 0
flann = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED)
matches = flann.knnMatch(track2.previous_track.raw_descriptor,track2.raw_descriptor, 2)

# store all the good matches as per Lowe's ratio test.
good = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        good.append(m)
        
#-- Draw matches
img1 = im2.image
img2 = im3.image

good



In [ ]:
img_matches = np.empty((max(img1.shape[0], img2.shape[0]), img1.shape[1]+img2.shape[1], 3), dtype=np.uint8)
cv2.drawMatches(img1, kp1, img2, kp2, good, img_matches, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
#-- Show detected matches
plt.imshow(img_matches)

In [ ]:
        # find keypoints for current track
        kp1 = track2.previous_track.raw_keypoints
        des1 = track2.previous_track.raw_descriptor

        # find keypoints for previous track
        kp2 = track2.raw_keypoints
        des2 = track2.raw_descriptor
            
        if type(des1) != np.float32:
            des1 = np.float32(des1)

        if type(des2) != np.float32:
            des2 = np.float32(des2)
                
        FLANN_INDEX_LSH = 6
        index_params= dict(algorithm = FLANN_INDEX_LSH,
                            table_number = 6, # 12
                            key_size = 12,     # 20
                            multi_probe_level = 1) #2
        search_params = dict(checks=50)   # or pass empty dictionary
        flann = cv2.FlannBasedMatcher(index_params,search_params)
        matches = flann.knnMatch(des1,des2, k=2)

        #Filter matches using the Lowe's ratio test
        ratio_thresh = 0.7
        good_matches = []
        for m,n in matches:
            if m.distance < ratio_thresh * n.distance:
                good_matches.append(m)